In [1]:
import sys

import pandas as pd
import numpy as np

import sklearn
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn import svm

from sklearn.metrics import mean_squared_error, r2_score

from sklearn import preprocessing

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


import joblib

In [2]:
filename = 'ginf.csv'
games = pd.read_csv('./data/' + filename)


In [3]:
games['result'] = games['fthg'] - games['ftag']

col = 'result'
conditions = [games[col] > 0 , games[col] == 0, games[col] < 0]
choices = [1 , 0 , -1]
games['result'] = np.select(conditions,choices,default=np.nan)



games.head()

,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n,result
0,UFot0hit/,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN,1.0
1,Aw5DflLH/,/soccer/germany/bundesliga-2011-2012/augsburg-...,True,2011-08-06,D1,2012,germany,FC Augsburg,SC Freiburg,2,2,2.36,3.60,3.40,NaN,NaN,NaN,NaN,0.0
2,bkjpaC6n/,/soccer/germany/bundesliga-2011-2012/werder-br...,True,2011-08-06,D1,2012,germany,Werder Bremen,Kaiserslautern,2,0,1.83,4.20,4.80,NaN,NaN,NaN,NaN,1.0
3,CzPV312a/,/soccer/france/ligue-1-2011-2012/paris-sg-lori...,True,2011-08-06,F1,2012,france,Paris Saint-Germain,Lorient,0,1,1.55,4.50,9.40,NaN,NaN,NaN,NaN,-1.0
4,GUOdmtII/,/soccer/france/ligue-1-2011-2012/caen-valencie...,True,2011-08-06,F1,2012,france,Caen,Valenciennes,1,0,2.50,3.40,3.45,NaN,NaN,NaN,NaN,1.0


In [4]:
games_prediction = games.iloc[:,[3,5,7,8,-1,-6,-7,-8]]
new_games = games_prediction.copy()
new_games['form1_short'] = new_games.apply (lambda row: 
                                      new_games[( ( (new_games['ht'] == row['ht']))
                                                 & (new_games['date'] < row['date']))].iloc[-3:]['result'].sum()
                                      -
                                      new_games[( ( (new_games['at'] == row['ht']))
                                                 & (new_games['date'] < row['date']))].iloc[-3:]['result'].sum()
                                      , axis=1)
new_games['form2_short'] = new_games.apply (lambda row: 
                                      -new_games[( ( (new_games['at'] == row['at']))
                                                 & (new_games['date'] < row['date']))].iloc[-3:]['result'].sum()
                                      +
                                      new_games[( ( (new_games['ht'] == row['at']))
                                                 & (new_games['date'] < row['date']))].iloc[-3:]['result'].sum()
                                      , axis=1)
new_games['form1_long'] = new_games.apply (lambda row: 
                                      new_games[( ( (new_games['ht'] == row['ht']))
                                                 & (new_games['date'] < row['date']))].iloc[-10:]['result'].sum()
                                      -
                                      new_games[( ( (new_games['at'] == row['ht']))
                                                 & (new_games['date'] < row['date']))].iloc[-10:]['result'].sum()
                                      , axis=1)
new_games['form2_long'] = new_games.apply (lambda row: 
                                      -new_games[( ( (new_games['at'] == row['at']))
                                                 & (new_games['date'] < row['date']))].iloc[-10:]['result'].sum()
                                      +
                                      new_games[( ( (new_games['ht'] == row['at']))
                                                 & (new_games['date'] < row['date']))].iloc[-10:]['result'].sum()
                                      , axis=1)


#new_games[( (new_games['at'] == 'Hamburg SV') & (new_games['date'] < '2011-05-06'))].iloc[-5:]['result'].sum()
new_games

,date,season,ht,at,result,odd_a,odd_d,odd_h,form1_short,form2_short,form1_long,form2_long
0,2011-08-05,2012,Borussia Dortmund,Hamburg SV,1.0,7.42,4.41,1.56,0.0,0.0,0.0,0.0
1,2011-08-06,2012,FC Augsburg,SC Freiburg,0.0,3.40,3.60,2.36,0.0,0.0,0.0,0.0
2,2011-08-06,2012,Werder Bremen,Kaiserslautern,1.0,4.80,4.20,1.83,0.0,0.0,0.0,0.0
3,2011-08-06,2012,Paris Saint-Germain,Lorient,-1.0,9.40,4.50,1.55,0.0,0.0,0.0,0.0
4,2011-08-06,2012,Caen,Valenciennes,1.0,3.45,3.40,2.50,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10107,2017-01-22,2017,Genoa,Crotone,0.0,8.00,4.35,1.97,-4.0,-3.0,-3.0,-12.0
10108,2017-01-22,2017,Chelsea,Hull,1.0,20.00,8.50,1.19,4.0,-3.0,13.0,-10.0
10109,2017-01-22,2017,AS Monaco,Lorient,1.0,11.50,6.24,1.32,3.0,1.0,11.0,-7.0
10110,2017-01-22,2017,Real Betis,Sporting Gijon,0.0,5.90,4.07,1.74,-1.0,-4.0,-2.0,-8.0


In [5]:
new_games[( ( (new_games['ht'] == 'Chelsea') | (new_games['at'] == 'Chelsea'))
                                                 & (new_games['date'] < '2017-01-22'))].iloc[-7:]

,date,season,ht,at,result,odd_a,odd_d,odd_h,form1_short,form2_short,form1_long,form2_long
9882,2016-12-11,2017,Chelsea,West Brom,1.0,14.00,6.50,1.29,6.0,1.0,8.0,-2.0
9897,2016-12-14,2017,Sunderland,Chelsea,-1.0,1.52,4.94,8.10,0.0,6.0,-5.0,10.0
9918,2016-12-17,2017,Crystal Palace,Chelsea,-1.0,1.56,4.50,7.20,-3.0,6.0,-6.0,10.0
9975,2016-12-26,2017,Chelsea,Bournemouth,1.0,11.00,5.30,1.44,6.0,0.0,12.0,-4.0
9989,2016-12-31,2017,Chelsea,Stoke City,1.0,16.00,6.85,1.25,6.0,0.0,13.0,-2.0
10003,2017-01-04,2017,Tottenham,Chelsea,1.0,3.10,3.21,2.78,4.0,6.0,8.0,14.0
10040,2017-01-14,2017,Leicester City,Chelsea,-1.0,1.85,3.70,5.10,0.0,4.0,-4.0,13.0


In [6]:


data = pd.DataFrame();
data['result'] = 0
for i, j in new_games.iterrows(): 
    if j['ht'] not in data.columns:
            data[j['ht']] = 0
    if j['at'] not in data.columns:
        data[j['at']] = 0
    if j['season'] not in data.columns:
        data[j['season']] = 0
    data = data.append({'result': j['result'], j['season']: 1, j['ht'] : 1 , 
                        j['at']: -1, 'form1_short' : j['form1_short'],'form2_short' : j['form2_short']
                       , 'form1_long' : j['form1_long'],'form2_long' : j['form2_long']
                       ,'odd_h' : j['odd_h'], 'odd_d' : j['odd_d'] , 'odd_a' : j['odd_a']}, ignore_index=True)

data


,result,Borussia Dortmund,Hamburg SV,2012,form1_long,form1_short,form2_long,form2_short,odd_a,odd_d,...,FC Ingolstadt 04,Las Palmas,Frosinone,Carpi,2017,Middlesbrough,Alaves,Crotone,Leganes,RB Leipzig
0,1.0,1.0,-1.0,1.0,0.0,0.0,0.0,0.0,7.42,4.41,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,3.40,3.60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,4.80,4.20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,9.40,4.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,3.45,3.40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10107,0.0,NaN,NaN,NaN,-3.0,-4.0,-12.0,-3.0,8.00,4.35,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN
10108,1.0,NaN,NaN,NaN,13.0,4.0,-10.0,-3.0,20.00,8.50,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
10109,1.0,NaN,NaN,NaN,11.0,3.0,-7.0,1.0,11.50,6.24,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
10110,0.0,NaN,NaN,NaN,-2.0,-1.0,-8.0,-4.0,5.90,4.07,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [7]:
data = data.replace(np.nan, 0)
data

,result,Borussia Dortmund,Hamburg SV,2012,form1_long,form1_short,form2_long,form2_short,odd_a,odd_d,...,FC Ingolstadt 04,Las Palmas,Frosinone,Carpi,2017,Middlesbrough,Alaves,Crotone,Leganes,RB Leipzig
0,1.0,1.0,-1.0,1.0,0.0,0.0,0.0,0.0,7.42,4.41,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.40,3.60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.80,4.20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.40,4.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.45,3.40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10107,0.0,0.0,0.0,0.0,-3.0,-4.0,-12.0,-3.0,8.00,4.35,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0
10108,1.0,0.0,0.0,0.0,13.0,4.0,-10.0,-3.0,20.00,8.50,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10109,1.0,0.0,0.0,0.0,11.0,3.0,-7.0,1.0,11.50,6.24,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10110,0.0,0.0,0.0,0.0,-2.0,-1.0,-8.0,-4.0,5.90,4.07,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
X = data.iloc[:,1:]
y = data.iloc[:,:1]
#y = np.ravel(y)
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=1)

(10112, 161)
(10112, 1)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123 , stratify=y )
#y_test = y_test.ravel();

In [11]:

pipeline = make_pipeline(KNeighborsClassifier())

hyperparameters = { 'kneighborsclassifier__n_neighbors' : [15, 40, 100],
                  'kneighborsclassifier__algorithm': ['auto', 'ball_tree', 'kd_tree'],
                  'kneighborsclassifier__weights': ['uniform','distance']}


clf = GridSearchCV(pipeline, hyperparameters, cv=10)

In [12]:
#clf = RandomForestClassifier(n_estimators=200)
#clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(30,20,3), random_state=1 , max_iter = 1000)
#clf = KNeighborsClassifier(n_neighbors=100, algorithm='auto');
#clf = RandomForestClassifier(n_estimators=400)
#clf = svm.SVC(kernel='linear')
clf.fit(X_train,y_train)

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sk

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sk

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sk

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sk

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sk

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sk

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sk

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('kneighborsclassifier',
                                        KNeighborsClassifier(algorithm='auto',
                                                             leaf_size=30,
                                                             metric='minkowski',
                                                             metric_params=None,
                                                             n_jobs=None,
                                                             n_neighbors=5, p=2,
                                                             weights='uniform'))],
                                verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'kneighborsclassifier__algorithm': ['auto',
                                                             'ball_tree',
                                                             'kd_

In [13]:
y_pred = clf.predict(X_test)

print(r2_score(y_test, y_pred))

print(mean_squared_error(y_test, y_pred))


-0.5455413658673505
1.1054713249835202


In [14]:
accuracy = clf.score(X_test, y_test)
#y_pred = clf.predict_proba(X_test)
#predict = clf.predict(X_test)
#y_total = y_train.count()
#y_positive = y_train.sum()
#print('The training set contains {} examples (games) of which {} are positives .'.format(y_total, y_positive))
print('The accuracy of classifying is {:.2f} %'.format(accuracy*100))

The accuracy of classifying is 52.14 %


In [ ]:
game = 100
labels = X_test.columns.values

'''
for x in range(len(X_test.iloc[game])): 
    if X_test.iloc[game][x] == 1:
        print(labels[x])
'''



In [ ]:
print(y_test.columns.values)
clf.predict([X_test.iloc[game]])

In [ ]:
y_test.iloc[game]

In [ ]:
joblib.dump(clf, './models/rfr_predict_winner.pkl')